# Soluzione 'The Bot Challenge' - Davide Pruscini (prushh)

## Engine
Al fine di risolvere i vari tasks, è stato necessario studiare il comportamento dell'engine e trovare:
 * i comandi a disposizione
 * come visualizzare/completare le quests
 * come chiudere la challenge

Per avviare il bot, messo a disposizione grazie ai tre file eseguibili (rispettivamente per Linux, MacOS o Windows), bisogna memorizzare il **TOKEN** all'interno di un file denominato `token.txt` e per interagire con esso basterà digitare, nella barra di ricerca di Telegram, il `<bot_username>` scelto in fase di registrazione. Per maggiori informazioni: 
[Come creare un bot](https://core.telegram.org/bots)

In fase di presentazione della challenge sono stati fatti alcuni spoiler per quanto riguarda i comandi disponibili:
 * **/status** - Mostra il numero di quests completate
 * **/quest0** - Permette di rispondere alla prima quest
 * **/quest1** - Permette di rispondere alla seconda quest
 * **/quest2** - Permette di rispondere alla terza quest
 
Oltre a questi non sono stati trovati altri comandi utilizzabili, si è passati quindi alla ricerca della parola chiave legata ad *Harry Potter*. Dopo innumerevoli tentativi collegati più che mai a *Piton*, ho deciso di cedere e guardare all'interno dell'eseguibile sperando di trovare qualcosa.

Fortunatamente è stato così, digitando la frase `giuro solennemente di non avere buone intenzioni` sono comparsi i tre bottoni che permettono la lettura delle quests, una dopo l'altra, soltanto se la precedente è stata completata. Si parlerà in seguito della loro risoluzione.

Non rimaneva che chiudere la challenge, per farlo è stato semplice, avendo visto recentemente la famosa saga è bastata una ricerca online: per mascherare la *Mappa del Malandrino* è sufficiente dire `fatto il misfatto`. Ovviamente ciò è possibile soltanto se tutte le quests sono state completate con successo.

## Sviluppo Bot
Come da direttive è stato usato il pacchetto [python-telegram-bot](https://github.com/python-telegram-bot/python-telegram-bot), all'interno del repository di GitHub è presente una cartella contenente degli [esempi](https://github.com/python-telegram-bot/python-telegram-bot/tree/master/examples) ed è proprio da lì che sono partito.
### Main

In [ ]:
def main():
    # Create the EventHandler and pass it your bot's token
    updater = Updater(TOKEN, use_context=True)
    dp = updater.dispatcher

    # Display authorization message
    bot_username = updater.bot.get_me()['username']
    print(f"{get_now()} Authorized on account {bot_username}")

    # Adding all the handler for the commands
    dp.add_handler(CommandHandler('status', status))
    dp.add_handler(CommandHandler('quest0', quest0))
    dp.add_handler(CommandHandler('quest1', quest1))
    dp.add_handler(CommandHandler('quest2', quest2))
    dp.add_handler(CommandHandler('quest3', quest3))
    dp.add_handler(CommandHandler('quest4', quest4))
    dp.add_handler(CommandHandler('quest5', quest5))
    dp.add_handler(MessageHandler(Filters.command, unknown))

    cmd_unlocks = ConversationHandler(
        entry_points=[MessageHandler(Filters.text, unlocks)],

        states={
            1: [MessageHandler(
                    Filters.regex('^(Quest 0|Quest 1|Quest 2|Quest 3|Quest 4|Quest 5)$'),
                    quest_choice),
                MessageHandler(Filters.text, quest_choice)],
        },

        fallbacks=[CommandHandler('cancel', cancel)]
    )
    dp.add_handler(cmd_unlocks)

    # Log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process
    # receives SIGINT, SIGTERM or SIGABRT.
    updater.idle()

    return 0

All'interno della funzione `main()` si va ad effettuare la creazione vera e propria del bot, si aggiungono gli handler per i comandi conosciuti e sconosciuti, viene inoltre utilizzato un `ConversationHandler` per interagire con ogni singolo utente.

Si procede all'avvio con l'istruzione seguente:
```python
updater.start_polling()
```

### Unlocks

In [ ]:
def unlocks(update, context):
    '''
    Unlock missions with the correct passphrase
    '''
    msg = update.message.text.lower()
    reply = "Non ho niente da dire..."

    if msg == passphrase:
        if 'quests' not in context.user_data.keys():
            context.user_data['quests'] = create_qts()
        reply_keyboard = [
            ['Quest 0', 'Quest 1', 'Quest 2'],
            ['Quest 3', 'Quest 4', 'Quest 5']]
        markup = ReplyKeyboardMarkup(reply_keyboard, resize_keyboard=True)
        update.message.reply_text(reply, reply_markup=markup)
        return 1
    elif msg == endphrase:
        if 'quests' in context.user_data.keys():
            quests = context.user_data['quests']
            if get_solved(quests) == NUM_QTS:
                reply = "Congratulazioni! Hai finito tutte le missioni..."
        else:
            reply = "Non hai completato tutte le missioni. Che peccato..."

    update.message.reply_text(reply)

Questo funzione rappresenta l'*entry_points* del `ConversationHandler`, viene richiamata ogni volta che s'invia un messaggio di testo grazie alla specifica `Filters.text`. Si confronta il messaggio inviato con la passphrase, se coincidono ed è la prima volta che è stata inserita si provvede alla creazione di un dizionario di dizionari all'interno dell'apposito spazio per l'utente:

```python
context.user_data['quests'] = create_qts()
```

Quest'ultimo conterrà tutte le quests, ogni *key* rappresenta una missione e sarà così composta:

```python
num_quest: {
    'text': quesito,
    'solution': soluzione,
    'solved': bool,
    'attemp': int
}
```

Viene inoltre creata e resa visibile la `ReplyKeyboardMarkup` contenente i bottoni, si ritorna 1 così da cambiare lo stato del `ConversationHandler`. Se invece è stata inserita l'endphrase si controlla che tutte le quests siano state completate.

In ogni caso viene inviato un messaggio di risposta.

### Quest choice

In [ ]:
def quest_choice(update, context):
    '''
    Bot core to interact with quests.
    '''
    msg = update.message.text.lower()
    quests = context.user_data['quests']

    if msg == endphrase:
        if get_solved(quests) == NUM_QTS:
            reply = "Congratulazioni! Hai finito tutte le missioni..."
            reply_markup = ReplyKeyboardRemove()
            update.message.reply_text(reply, reply_markup=reply_markup)
            return ConversationHandler.END
        reply = "Non hai completato tutte le missioni. Che peccato..."
    elif msg == 'quest 0':
        reply = check_choice(quests, 0)
    elif msg == 'quest 1':
        reply = check_choice(quests, 1)
    elif msg == 'quest 2':
        reply = check_choice(quests, 2)
    elif msg == 'quest 3':
        reply = check_choice(quests, 3)
    elif msg == 'quest 4':
        reply = check_choice(quests, 4)
    elif msg == 'quest 5':
        reply = check_choice(quests, 5)
    else:
        reply = "Non ho niente da dire..."

    update.message.reply_text(reply)

Ora siamo passati allo stato 1 del `ConversationHandler`, rappresentato da questa funzione. Si controlla il messaggio che l'utente ha inviato cercando di trovare un riscontro o con l'endphrase (dove in questo caso viene rimossa la `ReplyKeyboardMarkup` e conclusa la conversazione) o con una delle quests presenti.

Se la scelta corrisponde ad una quest si procede con la seguente funzione che restituisce un messaggio a seconda dei casi.

Verifica inoltre se tutte le quests precedenti sono state completate o meno.

In [ ]:
def check_choice(qts: dict, n: int) -> str:
    '''
    Check if the question can be answered.
    '''
    if qts[n]['solved']:
        return "Hai già completato questa quest..."

    for idx in range(0, n):
        if not qts[idx]['solved']:
            return "Devi completare la quest precedente..."
    return qts[n]['text']

### Quest-i

In [ ]:
def quest0(update, context):
    '''Command for quest0.'''
    reply = check_qts(context, 0)
    update.message.reply_text(reply)

Per ogni quest viene richiamata la stessa funzione, si controlla la correttezza della risposta inviando all'utente un apposito messaggio.

Si aggiorna inoltre il numero di tentativi effettuati per una determinata quest.

In [ ]:
def check_qts(context, n: int) -> str:
    '''
    Check if the answer is correct.
    '''
    reply = "Risposta non corretta..."
    if 'quests' not in context.user_data.keys():
        return reply

    qts = context.user_data['quests'][n]
    if qts['solved']:
        return reply

    args = " ".join(context.args)
    if _cast_arg(args) == qts['solution']:
        context.user_data['quests'][n]['solved'] = True
        return "Risposta corretta! Quest completata!"

    context.user_data['quests'][n]['attemp'] += 1
    reply += f" Tentativi effettuati: {qts['attemp']}"
    return reply

### Status

In [ ]:
def status(update, context):
    '''Show number of completed quests.'''
    if 'quests' in context.user_data.keys():
        quests = context.user_data['quests']
        solved_qts = get_solved(quests)
        reply = f"Hai completato {solved_qts} quest su {NUM_QTS}"
    else:
        reply = f"Hai completato 0 quest su {NUM_QTS}"
    update.message.reply_text(reply)

Mette a conoscenza l'utente su quante quests ha completato e quante ce ne sono complessivamente.

### Unknown

In [ ]:
def unknown(update, context):
    '''Reply to all unrecognized commands.'''
    update.message.reply_text("Comando non valido...")

Ogni qual volta l'utente invii un comando sconosciuto si risponde semplicemente come sopra.

## Risoluzione Quests
Di seguito le funzioni create per risolvere le quest, *N* rappresenta un numero intero random preso in un determinato range, *DATE* rappresenta una data nel formato `%Y/%m/%d`.
### 0. Calcola la somma dei multipli di 3 e 5 fino a *N*

In [ ]:
def _quest0(last: int) -> int:
    sum_ = 0
    for elm in range(last):
        if elm % 3 == 0 or elm % 5 == 0:
            sum_ += elm

    return sum_

### 1. Calcola la somma dei numeri dispari della serie di fibonacci fino all'*N*-esimo

In [ ]:
def _quest1(n: int) -> int:
    n += 2
    sum_ = -1

    a, b = 0, 1
    while n > 0:
        if a % 2 != 0:
            sum_ += a
        a, b = b, a + b
        n -= 1

    return sum_

### 2. Decodifica la stringa: *SGVsbG8sIFB56K6t57uD6JClIQ==%!(EXTRA int=21)*

In [ ]:
def _quest2(b64: str) -> str:
    return b64decode(b64).decode('utf-8')

### 3. Trova la cifra decimale numero *N* del π

In [ ]:
def _quest3(nth: int) -> int:
    idx = nth + 1
    tmp = "%.48f" % pi
    return int(tmp[idx])

### 4. Trova una differente rappresentazione: *DATE*

In [ ]:
def _quest4(date: str) -> int:
    return int(datetime.strptime(date, "%Y/%m/%d").timestamp())

### 5. Conosci qualche alfabeto? Prova a fare lo spelling: *PyBootCamp*

In [ ]:
def _quest5(text: str) -> str:
    return nato(text)